In [1]:
import os
import pandas as pd
from nilearn import image as img
from nilearn import plotting as plt
import matplotlib.pyplot as plt
import nibabel as nib

%matplotlib inline

/home/nima/.local/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
base_dir = os.path.expanduser('~') + '/Desktop/study_forrest_rsa'
fmri_dir = base_dir + '/derivatives/fmriprep'
out_dir = base_dir + '/derivatives/output'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [3]:
sub_list = ['01', '02', '03', '04', '06', '09', '10', '19']
run_list = ['%.1d' % j for j in range(1, 9)]

In [4]:
def load_preproc(sub_num, run_num, fmri_dir, out_dir):
    # data paths
    func_file = fmri_dir + f'/sub-{sub_num}/ses-movie/func/sub-{sub_num}_ses-movie_task-movie_run-{run_num}_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz'
    conf_file = fmri_dir + f'/sub-{sub_num}/ses-movie/func/sub-{sub_num}_ses-movie_task-movie_run-{run_num}_desc-confounds_regressors.tsv'
    mask_file = fmri_dir + f'/sub-{sub_num}/ses-movie/func/sub-{sub_num}_ses-movie_task-movie_run-{run_num}_space-MNI152NLin2009cAsym_res-2_desc-brain_mask.nii.gz'
    
    out_path = out_dir + f'/sub-{sub_num}'
    
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    
    # load confound variables from fMRIPrep
    confound_df = pd.read_csv(conf_file, delimiter='\t')
    confound_vars = ['white_matter', 'csf', 'trans_x', 'trans_x_derivative1', 'trans_y', 'trans_y_derivative1', 
                     'trans_z', 'trans_z_derivative1', 'rot_x', 'rot_x_derivative1', 'rot_y', 'rot_y_derivative1', 'rot_z',
                     'rot_z_derivative1', 'global_signal', 'global_signal_derivative1', 'a_comp_cor_00', 'a_comp_cor_01']
    confound_df = confound_df[confound_vars]
    
    # drop the first four rows because it contains NaNs
    drop_confound_df = confound_df.loc[5:]
    confounds_matrix = drop_confound_df#.values()
    print('Shape of the confound matrix: \t', drop_confound_df.shape)
    
    # load the normalized BOLD file, drop the first four volumes too
    raw_func_img = img.load_img(func_file)
    func_img = raw_func_img.slicer[:, :, :, 5:]
    print('Shape of the functional image: \t',func_img.shape)
    
    if drop_confound_df.shape[0] != func_img.shape[3]:
        raise ValueError('ValueError exception thrown')
    
    # set some constants for all subs / runs
    high_pass= 0.009
    low_pass = 0.08
    t_r = 2
    
    clean_img = img.clean_img(func_img, confounds=confounds_matrix, detrend=True, standardize=True, 
                              low_pass=low_pass, high_pass=high_pass, t_r=t_r, mask_img=mask_file)
    
    clean_img.to_filename(out_path + f'/sub-{sub_num}_run-{run_num}_cleaned.nii.gz')

In [ ]:
for sub in sub_list:
    for run in run_list:
        load_preproc(sub_num=sub, run_num=run, fmri_dir=fmri_dir, out_dir=out_dir)

Shape of the confound matrix: 	 (446, 18)
Shape of the functional image: 	 (97, 115, 97, 446)
Shape of the confound matrix: 	 (436, 18)
Shape of the functional image: 	 (97, 115, 97, 436)
Shape of the confound matrix: 	 (433, 18)
Shape of the functional image: 	 (97, 115, 97, 433)
Shape of the confound matrix: 	 (483, 18)
Shape of the functional image: 	 (97, 115, 97, 483)
Shape of the confound matrix: 	 (457, 18)
Shape of the functional image: 	 (97, 115, 97, 457)
Shape of the confound matrix: 	 (434, 18)
Shape of the functional image: 	 (97, 115, 97, 434)
Shape of the confound matrix: 	 (537, 18)
Shape of the functional image: 	 (97, 115, 97, 537)
Shape of the confound matrix: 	 (333, 18)
Shape of the functional image: 	 (97, 115, 97, 333)
Shape of the confound matrix: 	 (446, 18)
Shape of the functional image: 	 (97, 115, 97, 446)
Shape of the confound matrix: 	 (436, 18)
Shape of the functional image: 	 (97, 115, 97, 436)
Shape of the confound matrix: 	 (433, 18)
Shape of the funct